In [2]:
text="""경마장에 있는 말이 뛰고 있다. 그의 말이 법이다. 가는 말이 고와야 오는 말이 곱다."""

In [3]:
import tensorflow as tf
from keras_preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]
print(text)
print(encoded) # 빈도 수에 따라 번호 부여(빈도가 잦을수록 작은 숫자)

경마장에 있는 말이 뛰고 있다. 그의 말이 법이다. 가는 말이 고와야 오는 말이 곱다.
[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]


In [4]:
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [5]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [6]:
from typing import Sequence
sequences = list()
for line in text.split('.') :
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)) :
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('훈련 데이터의 개수 :',len(sequences))

훈련 데이터의 개수 : 11


In [7]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [8]:
print(max(len(I) for I in sequences))

6


In [9]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=6, padding='pre')

In [10]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [11]:
# x, y로 시퀀스 분리
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [12]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [13]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [14]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [15]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [16]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=5))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 2.4458 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 2.4318 - accuracy: 0.1818
Epoch 3/200
1/1 [==============================] - 0s 13ms/step - loss: 2.4172 - accuracy: 0.2727
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 2.4021 - accuracy: 0.3636
Epoch 5/200
1/1 [==============================] - 0s 12ms/step - loss: 2.3864 - accuracy: 0.3636
Epoch 6/200
1/1 [==============================] - 0s 12ms/step - loss: 2.3699 - accuracy: 0.3636
Epoch 7/200
1/1 [==============================] - 0s 13ms/step - loss: 2.3528 - accuracy: 0.4545
Epoch 8/200
1/1 [==============================] - 0s 14ms/step - loss: 2.3349 - accuracy: 0.4545
Epoch 9/200
1/1 [==============================] - 0s 17ms/step - loss: 2.3161 - accuracy: 0.4545
Epoch 10/200
1/1 [==============================] - 0s 11ms/step - loss: 2.2966 - accuracy: 0.4545
Epoch 11/200
1/1 

In [17]:
def sentence_generation(model, t, current_word, n) :
  init_word = current_word
  sentence = ''
  for _ in range(n) :
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    #result = model.predict_classes(encoded, verbose=0) #tf2.6 deprecated
    y_prob = model.predict(encoded, verbose=0)
    result = y_prob.argmax(axis=-1)
    for word, index in t.word_index.items() :
      if index == result :
        break

    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word
  sentence = init_word + sentence
  return sentence

In [18]:
print(sentence_generation(model, t, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [19]:
print(sentence_generation(model, t, '그의', 2))

그의 말이 법이다


In [20]:
print(sentence_generation(model, t, '가는', 5))

가는 말이 고와야 오는 말이 곱다


In [21]:
print(sentence_generation(model, t, '경마장에', 7))

경마장에 있는 말이 뛰고 있다 있다 곱다 있다
